In [16]:
import cv2
import numpy as np
from matplotlib import pyplot as plt 

In [17]:
cap = cv2.VideoCapture('Robots.mp4')    

In [18]:
# Read first frame
ret, frame = cap.read()
if not ret:
    print("Failed to read video")
    exit()

# Resize first frame
frame_small = cv2.resize(frame, None, fx=0.5, fy=0.5, interpolation=cv2.INTER_AREA)

# Select ROI (on resized frame)
roi = cv2.selectROI("Select Object", frame_small, fromCenter=False, showCrosshair=True)
x, y, w, h = roi
cv2.destroyWindow("Select Object")

# Convert to grayscale
old_gray = cv2.cvtColor(frame_small, cv2.COLOR_BGR2GRAY)

# Detect feature points in ROI
mask = np.zeros_like(old_gray)
mask[y:y+h, x:x+w] = 255
p0 = cv2.goodFeaturesToTrack(old_gray, mask=mask, maxCorners=100, qualityLevel=0.3, minDistance=7)

# Lucas-Kanade parameters
lk_params = dict(winSize=(15, 15), maxLevel=2,
                 criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))

# Random colors for feature tracks
color = np.random.randint(0, 255, (100, 3))
mask_draw = np.zeros_like(frame_small)

# Loop
ret = True
while ret:
    ret, frame = cap.read()
    if not ret:
        break

    # Resize frame
    frame_small = cv2.resize(frame, None, fx=0.5, fy=0.5, interpolation=cv2.INTER_AREA)
    frame_gray = cv2.cvtColor(frame_small, cv2.COLOR_BGR2GRAY)

    # Optical flow
    if p0 is not None:
        p1, st, err = cv2.calcOpticalFlowPyrLK(old_gray, frame_gray, p0, None, **lk_params)

        if p1 is not None:
            good_new = p1[st == 1]
            good_old = p0[st == 1]

            # Draw tracks
            for i, (new, old) in enumerate(zip(good_new, good_old)):
                a, b = new.ravel()
                c, d = old.ravel()
                a, b, c, d = int(a), int(b), int(c), int(d)  # <-- cast to int
                mask_draw = cv2.line(mask_draw, (a, b), (c, d), color[i].tolist(), 2)
                frame_small = cv2.circle(frame_small, (a, b), 4, color[i].tolist(), -1)


            # Update bounding box around tracked points
            if len(good_new) > 0:
                x_min, y_min = np.min(good_new, axis=0).astype(int)
                x_max, y_max = np.max(good_new, axis=0).astype(int)
                frame_small = cv2.rectangle(frame_small, (x_min, y_min), (x_max, y_max), (0, 255, 0), 2)


            # Update points
            p0 = good_new.reshape(-1, 1, 2)

    # Update reference frame
    old_gray = frame_gray.copy()

    # Show result
    img = cv2.add(frame_small, mask_draw)
    cv2.imshow("image", img)

    if cv2.waitKey(20) & 0xFF == 27:  # press ESC to exit
        break

cap.release()
cv2.destroyAllWindows()


Select a ROI and then press SPACE or ENTER button!
Cancel the selection process by pressing c button!
